<a href="https://colab.research.google.com/github/20v100/Coursera_Capstone/blob/master/TorontoNeighborhoodsPrelim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Exploration and clustering of Toronto Neighborhoods](#scrollTo=r6Ypu_zK0Bu-)

>>[Import dependencies and load the postal codes](#scrollTo=NGHtOhQd1IBq)

>>[Get Foursquare data for Toronto Neighboorhods](#scrollTo=642Hmhobn3As)

>>>[Select borough that contains "Toronto"](#scrollTo=RUv3z2d71rSK)

>>>[Run the fuction on several neighborhoods](#scrollTo=PdkE-PQVr7Td)

>>[Wrangle data for K-Means Clusteting](#scrollTo=8qNvdnMt2r7A)

>>>[One Hot Encoding and Aggragate Stats](#scrollTo=1kILFcvn2Paj)

>>[K-Means Clutering](#scrollTo=Ti_ZXH0j36Tr)

>>>[Running K-means](#scrollTo=NTJUY-BW5tBg)

>>>[Merging Clusters and Toronto Borough](#scrollTo=CexeErH5DDLT)

>>>>[Filtering the Toronto Borough withthe 10 most popular venues](#scrollTo=8VJVk8duDV3g)

>>>[Maping the clusters](#scrollTo=C73aJk-QHtfU)

>>>[Reviewing the cluster meaning](#scrollTo=i-LJCz3PA3-V)

>>>[Findings on Clusters](#scrollTo=9brPSAvcCjh5)



#Exploration and clustering of Toronto Neighborhoods

**Vincent Gauthier**

##Import dependencies and load the postal codes


In [2]:
# import the library

# !pip install geopy
# from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

    100% |████████████████████████████████| 92kB 3.4MB/s 
Folium installed
Libraries imported.


In [3]:
#Four Square Credentials
CLIENT_ID = 'Z4G102XHUFAUSKXKLFW01UEVDEO3XIDLWNTGEWAKJDNN21JW' # your Foursquare ID
CLIENT_SECRET = 'TFQ4CBEKWNASZD4FROH4NHUS5PWSGRT2A14RWA3UFRFC54EU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z4G102XHUFAUSKXKLFW01UEVDEO3XIDLWNTGEWAKJDNN21JW
CLIENT_SECRET:TFQ4CBEKWNASZD4FROH4NHUS5PWSGRT2A14RWA3UFRFC54EU


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Load Toronto Neighborhoods file mapped to postal coded and lat long
df_pc = pd.read_csv('drive/My Drive/torontoPostalCodeWtLogLat.csv')
df_pc.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:
search_query = ''
radius = 500
print(search_query + ' .... OK!')

 .... OK!


## Get Foursquare data for Toronto Neighboorhods

###Select borough that contains "Toronto"

In [7]:
# Selecting boroughs that contain "Toronto"
df_tor = df_pc[df_pc['Borough'].str.contains("Toronto")]
df_tor.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [8]:
print('Neighborhoods that we will explore: ', df_tor.shape[0])

Neighborhoods that we will explore:  38


###**Run** the fuction on several neighborhoods

In [0]:
#Function that explore **a** neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
tor_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [11]:
# Let's review the shape of the results
print(tor_venues.shape)
tor_venues.head()

(836, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Williamson Road Playground,43.674716,-79.297338,Playground
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [12]:
# Let's check how many venues were returned per neighborhoods
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16


In [13]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 187 uniques categories.


##Wrangle data for K-Means Clusteting

###One Hot Encoding and Aggragate Stats

In [14]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

tor_onehot = tor_onehot.drop('Neighborhood', 1)


# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Review the shape of tor_onehot
tor_onehot.shape

(836, 187)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [16]:
tor_venues.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [17]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
3,Business reply mail Processing Centre969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000


In [18]:
#Confirm the shape
tor_grouped.shape

(38, 187)

In [19]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0           Steakhouse  0.10
1                 Café  0.10
2                Hotel  0.07
3  American Restaurant  0.07
4     Asian Restaurant  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.10
1  Seafood Restaurant  0.07
2                Café  0.07
3      Farmers Market  0.07
4              Bakery  0.07


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0            Coffee Shop  0.15
1         Breakfast Spot  0.10
2  Performing Arts Venue  0.10
3                   Café  0.10
4           Climbing Gym  0.05


----Business reply mail Processing Centre969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2    Pizza Place  0.06
3     Restaurant  0.06
4  Burrito Place  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0

##K-Means Clutering

###Running K-means

In [20]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

###Merging Clusters and Toronto Borough

####Filtering the Toronto Borough withthe 10 most popular venues

In [0]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
# # Now let's create the new dataframe and display the top 10 venues for each neighborhood.
# num_top_venues = 10

# indicators = ['st', 'nd', 'rd']

# # create columns according to number of top venues
# columns = ['Neighborhood']
# for ind in np.arange(num_top_venues):
#     try:
#         columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
#     except:
#         columns.append('{}th Most Common Venue'.format(ind+1))

# # create a new dataframe
# neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
# neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

# for ind in np.arange(manhattan_grouped.shape[0]):
#     neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [23]:
tor_merged = tor_grouped

# add clustering labels
tor_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = pd.merge(tor_merged, tor_venues, on='Neighborhood', how='outer')

tor_merged.head() # check the last columns!

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,1,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
1,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,1,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650609,-79.386280,Concert Hall
2,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,1,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
3,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,1,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
4,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,1,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse


###Maping the clusters

In [24]:
# create map

latitude  = 43.6532
longitude = -79.3832

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Venue Latitude'], tor_merged['Venue Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###Reviewing the cluster meaning

In [25]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar,43.646676,-79.374822,Steakhouse
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,Sony Centre for the Performing Arts,43.646292,-79.376022,Concert Hall
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,D.W. Alexander,43.648333,-79.373826,Cocktail Bar
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,Sukhothai,43.648487,-79.374547,Thai Restaurant
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,St. Lawrence Market (South Building),43.648778,-79.371637,Farmers Market
37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,C'est What,43.648467,-79.373377,Beer Bar
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,Berczy Park,43.648048,-79.375172,Park
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.644771,-79.373306,St. Lawrence Market (North Building),43.648793,-79.371945,Farmers Market


In [26]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
1,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650609,-79.386280,Concert Hall
2,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
3,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
4,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
5,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
6,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
7,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Momofuku Noodle Bar,43.649366,-79.386217,Noodle House
8,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,Assembly Chef's Hall,43.650579,-79.383412,Food Court
9,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,1,43.650571,-79.384568,M Square Coffee Co,43.651218,-79.383555,Coffee Shop


In [36]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,43.711695,-79.416936,Dr.Paul Hodges MIP,43.710634,-79.415810,Health & Beauty Service
603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue


In [37]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,43.689574,-79.38316,Reds,43.692816,-79.384504,Restaurant
561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,43.689574,-79.38316,Moore Park Tennis Club,43.693289,-79.382900,Tennis Court
562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,43.689574,-79.38316,Moorevale Park,43.693610,-79.383465,Playground


In [38]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Wine Bar,Yoga Studio,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.696948,-79.411307,Kay Gardner Beltline Trail,43.700726,-79.410101,Trail
408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.696948,-79.411307,Forest Hill Road Park,43.697945,-79.406605,Park
409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.696948,-79.411307,Nikko Sushi Japenese Restaurant,43.700443,-79.407957,Sushi Restaurant
410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.696948,-79.411307,Oliver jewelry,43.700374,-79.407644,Jewelry Store
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


###Findings on Clusters

Cluster 0 "High End Experiences" : Neighbourhoods offering high-end experiences venues (steakhouses, cocktail bars, Farmers Markets, etc) as well as museums and stadiums.

Cluster 1 "International Neighbourhoods" : Neighbourhoods offering more affordable restaurants with international cuisines (Thai, Chinese, Greek etc.).

Cluster 2 "Low Density Venues" : Low density venues type Neighbourhoods.

Cluster 3 "Low Density Outdoor Activities" :  Low density volume outdoor leisure.

Cluster 4 "Outdoor Activities" : These neighbourhoods offer mainly outdoor leisure.